<a href="https://colab.research.google.com/github/shauncohere/cohere_rag-c/blob/main/cohere_rag_citations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qdrant-client
!pip install cohere langchain qdrant-client tfds-nightly

## QnA Bot with citations for an airbnB in the canary islands

In [ ]:
from langchain import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from IPython.display import display, Markdown, Latex
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.llms import Cohere
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
import textwrap as tr
import random
import os

os.environ["COHERE_API_KEY"] = ''

# Populate Text Embeddings into a VectorDB

In [ ]:
# save the embeddings in a DB that is persisten
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
loader = TextLoader("./facts.txt")
document=loader.load()
docs=document[0].page_content
text_splitter = CharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separator ='\n\n',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    add_start_index = True,
)
docs = text_splitter.create_documents([docs])
texts = [doc.page_content for doc in docs]


# Define the embeddings model
embeddings = CohereEmbeddings(model = 'embed-multilingual-v2.0')

metadata=[{"source": text} for text in texts]

# Embed the documents and store in index
vector_store = Qdrant.from_texts(texts, embeddings, location=":memory:",metadatas=metadata, collection_name="summaries", distance_func="Dot")


## Query the VectorDB return Generation and Citations

In [ ]:
retriever=vector_store.as_retriever()


prompt_template="""You are Delores, an AI customer service agent that talks to guests staying in an AirBnB in the canary islands.
Answer the questions using the facts provided. You always uses pleasantries and polite greetings like "Good morning"
and "Good afternoon". Use the following pieces of context to answer the users question.
Take note of the sources and include them in the answer in the format: "SOURCES: source1 source2", use "SOURCES" in capital letters regardless of the number of sources.
If you don't know the answer, just say that "I don't know", don't try to make up an answer.
----------------
{summaries}"""

messages = [
    SystemMessagePromptTemplate.from_template(prompt_template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)

chain_type_kwargs = {"prompt": prompt}

llm=Cohere(model="command-nightly")


chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def print_result(result):
  output_text = f"""### Question:
  {query}
  ### Answer:
  {result['answer']}
  ### Sources:
  {result['sources']}
  ### All relevant sources:
  {' '.join(list(set([doc.metadata['source'] for doc in result['source_documents']])))}
  """
  return(output_text)

In [ ]:
query = "Is there a bbq grill at the property?"
result = chain(query)
display(Markdown(print_result(result)))

### Question: 
  Is there a bbq grill at the property?
  ### Answer: 
   Good afternoon! Yes, we have a designated beachfront dining area and a barbecue facility for our guests. You can savor your meals while taking in the beach atmosphere or enjoy a delightful barbecue with friends and family.

  ### Sources: 
  Q: Is there a beachfront dining area or barbecue facility?
  ### All relevant sources:
  Q: Is there a beachfront dining area or barbecue facility?
A: Yes, we have a designated beachfront dining area and a barbecue facility for our guests. You can savor your meals while taking in the beach atmosphere or enjoy a delightful barbecue with friends and family. Q: Is there a beach bar or beachside lounge area for relaxation?
A: Certainly! We have a beach bar and lounge area exclusively for our guests. You can unwind with a refreshing drink or find a cozy spot to relax, read a book, or simply soak up the sun. Q: Are there beachside cafes or restaurants within walking distance?
A: Yes, there are several beachside cafes and restaurants just a short stroll away from our Airbnb. You can indulge in delicious local cuisine or enjoy a refreshing drink while admiring the beach views. Q: Are there any beachfront water activities or rentals available nearby?
A: Yes, there are beachfront water activity centers close to our Airbnb. You can rent jet skis, go on a thrilling parasailing adventure, or even book a guided snorkeling tour to explore the underwater beauty.
  

## Custom QnA with Citations

In [ ]:
query = input("How can I help?" + '\n')
result = chain(query)
display(Markdown(print_result(result)))

How can I help?
What is there to do at the property?


### Question: 
  What is there to do at the property?
  ### Answer: 
   There are many activities that you can do at the property. You can rent jet skis, go on a thrilling parasailing adventure, or even book a guided snorkeling tour to explore the underwater beauty. There is also a beach bar and lounge area exclusively for guests. You can unwind with a refreshing drink or find a cozy spot to relax, read a book, or simply soak up the sun. The property is also perfect for long walks or invigorating jogs along the shoreline. You can relish the fresh ocean breeze and take in the scenic beauty of the Canary Islands as you exercise. There is also a designated beachfront dining area and a barbecue facility for guests. You can savor your meals while taking in the beach atmosphere or enjoy a delightful barbecue with friends and family.
  ### Sources: 
  
  ### All relevant sources:
  Q: Are there any beachfront water activities or rentals available nearby?
A: Yes, there are beachfront water activity centers close to our Airbnb. You can rent jet skis, go on a thrilling parasailing adventure, or even book a guided snorkeling tour to explore the underwater beauty. Q: Is there a beach bar or beachside lounge area for relaxation?
A: Certainly! We have a beach bar and lounge area exclusively for our guests. You can unwind with a refreshing drink or find a cozy spot to relax, read a book, or simply soak up the sun. Q: Is there a beachfront dining area or barbecue facility?
A: Yes, we have a designated beachfront dining area and a barbecue facility for our guests. You can savor your meals while taking in the beach atmosphere or enjoy a delightful barbecue with friends and family. Q: Can I take long walks or go jogging on the beach?
A: Certainly! Our beachfront location is perfect for long walks or invigorating jogs along the shoreline. You can relish the fresh ocean breeze and take in the scenic beauty of the Canary Islands as you exercise.
  